In [ ]:
from grad_val import GradVal
from random import random

In [ ]:
model_size = 3
weights = [GradVal(random()) for _ in range(model_size)]
params = [GradVal(random()) for _ in range(model_size)]
out = [None for _ in range(model_size)]

for i, (w, p) in enumerate(zip(weights, params)):
    out[i] = (w * p).relu()

final = out[0] + out[1] + out[2]

In [ ]:
final.visualize()

In [ ]:
final.backward()
final.visualize()

In [1]:
from neural_net import *
from optimizer import GDOptimizer
from loss import MSELoss

BATCH_SIZE = 32

y_true = np.array([GradVal(random()) for _ in range(BATCH_SIZE)])
x = np.array([[GradVal(random()) for _ in range(8)] for _ in range(BATCH_SIZE)])

pipeline = [BiasedLinearLayer(8, 16), ReluLayer(), BiasedLinearLayer(16, 8), ReluLayer(), BiasedLinearLayer(8, 1)]
model = Model(pipeline=pipeline)
optimizer = GDOptimizer(model.parameters(), lr=1e-4)
loss = MSELoss()

y_pred = model.forward(x)
loss_grad = loss.loss(y_pred=y_pred, y_true=y_true)
loss_grad.backward()
optimizer.optimize()
loss_grad.zero_grad()


In [2]:
y_pred = model.forward(x)
loss_grad = loss.loss(y_pred=y_pred, y_true=y_true)
loss_grad.backward()
optimizer.optimize()
y_pred

array([24.612711457690214, 27.497397509711437, 19.365334760250317,
       20.7403452364392, 27.643250769148825, 29.01142505453139,
       20.460307780618127, 18.795140350595762, 26.284085694258657,
       21.751018757315947, 28.00827391940376, 17.034882787797656,
       19.26233852673598, 28.134702485591912, 21.595585415915256,
       21.433826312046783, 22.975873858832973, 20.460043331328716,
       20.76834572145255, 25.849794161662658, 23.35569263175384,
       22.192830520754516, 23.463734000176007, 15.49748699050049,
       31.091440723057058, 25.225651509053435, 24.496879444667357,
       23.801496720512432, 22.916731932939225, 22.922777977074112,
       28.620700677950268, 27.596678513502283], dtype=object)

In [3]:
loss_grad.val

537.4289285088021

In [4]:
base = []
seen = set()

def search(v):
    if v in seen:
        return
    seen.add(v)
    
    if not v.ancestors:
        base.append(v)
    
    for ancestor in v.ancestors:
        search(ancestor)
        
search(loss_grad)
print([k.gradient for k in base if k.gradient != 0])
loss_grad.zero_grad()

[]